In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("./emma-export-1638657-2023-12-08T08_36_19.053Z-19584.xlsx")

In [3]:
df.head()

,ID,Date,Amount,Account,Bank,Currency,Category,Type,Tags,Counterparty,Custom Name,Merchant,Additional details,Notes,Linked transaction ID
0,f55b36f6f94be05a6ce8be2f1a3f21e8,2023-12-06,-25.00,SELECT ACCOUNT,Natwest,GBP,Personal Care,Direct Debit,NaN,JD SPORTS GYMS,NaN,JD Sports,NaN,NaN,NaN
1,35bc00817e7dfbf97297b1547ff0b82a,2023-12-05,-51.56,SELECT ACCOUNT,Natwest,GBP,Transport,Purchase,NaN,ASDA PETROL BRADFORD GB,NaN,Asda Petrol Station,NaN,NaN,NaN
2,d6e7ca43f574e6a7e293c04ad07adafc,2023-12-04,-2.80,SELECT ACCOUNT,Natwest,GBP,Groceries,Purchase,NaN,SAINSBURYS S/MKTS LIVERSEDGE GB,NaN,Sainsbury's,NaN,NaN,NaN
3,7229bf9fa20593e2a781a5b33b5239ca,2023-12-04,-1.79,SELECT ACCOUNT,Natwest,GBP,Groceries,Purchase,NaN,FARMFOODS 405 DEWSBURY GB,NaN,Farmfoods,NaN,NaN,NaN
4,fcef157f83a1c608ca8da88c4f899b4f,2023-12-04,-3.75,SELECT ACCOUNT,Natwest,GBP,Eating Out,Purchase,NaN,TAMP & STEAM COFFEE HO LIVERSEDGE GB,NaN,NaN,NaN,NaN,NaN


In [126]:
aug_dec_23 = df[df['Date'].between('2023-08-01', '2023-12-08')]

In [127]:
aug_dec_23 = aug_dec_23.drop(aug_dec_23[aug_dec_23['Category']=='Excluded'].index)

In [128]:
def add_income_category(row):
    if row['Amount']>0:
        row['Category'] = row['Category'] + '_in'
    return row
aug_dec_23 = aug_dec_23.apply(add_income_category, axis=1)

In [129]:
import plotly.express as px

In [145]:
cat_map = {i:idx for idx, i in enumerate(set(aug_dec_23['Category']))}
cat_map = cat_map | {"in":len(cat_map)}
cat_map

{'Charity_in': 0,
 'Groceries': 1,
 'Housing': 2,
 'Personal Care': 3,
 'Transport_in': 4,
 'Holidays_in': 5,
 'Shopping': 6,
 'Income_in': 7,
 'Cash': 8,
 'Education': 9,
 'Housing_in': 10,
 'Eating Out': 11,
 'Transport': 12,
 'Family_in': 13,
 'Bills': 14,
 'Charity': 15,
 'Entertainment': 16,
 'Family': 17,
 'Eating Out_in': 18,
 'General': 19,
 'Holidays': 20,
 'Shopping_in': 21,
 'General_in': 22,
 'in': 23}

In [146]:
def apply_source_target(row):
    if row['Amount']>0:
        # income
        row['source'] = cat_map[row['Category']]
        # centre of sankey
        row['target'] = cat_map['in']
    else:
        # spending
        row['target'] = cat_map[row['Category']]
        # centre of sankey
        row['source'] = cat_map['in']
    return row

In [147]:
aug_dec_23['target'] = ""
aug_dec_23['source'] = ""

In [148]:
aug_dec_23 = aug_dec_23.apply(lambda x:apply_source_target(x), axis=1)

In [149]:
aug_dec_23.head()

,ID,Date,Amount,Account,Bank,Currency,Category,Type,Tags,Counterparty,Custom Name,Merchant,Additional details,Notes,Linked transaction ID,target,source
0,f55b36f6f94be05a6ce8be2f1a3f21e8,2023-12-06,-25.00,SELECT ACCOUNT,Natwest,GBP,Personal Care,Direct Debit,NaN,JD SPORTS GYMS,NaN,JD Sports,NaN,NaN,NaN,3,23
1,35bc00817e7dfbf97297b1547ff0b82a,2023-12-05,-51.56,SELECT ACCOUNT,Natwest,GBP,Transport,Purchase,NaN,ASDA PETROL BRADFORD GB,NaN,Asda Petrol Station,NaN,NaN,NaN,12,23
2,d6e7ca43f574e6a7e293c04ad07adafc,2023-12-04,-2.80,SELECT ACCOUNT,Natwest,GBP,Groceries,Purchase,NaN,SAINSBURYS S/MKTS LIVERSEDGE GB,NaN,Sainsbury's,NaN,NaN,NaN,1,23
3,7229bf9fa20593e2a781a5b33b5239ca,2023-12-04,-1.79,SELECT ACCOUNT,Natwest,GBP,Groceries,Purchase,NaN,FARMFOODS 405 DEWSBURY GB,NaN,Farmfoods,NaN,NaN,NaN,1,23
4,fcef157f83a1c608ca8da88c4f899b4f,2023-12-04,-3.75,SELECT ACCOUNT,Natwest,GBP,Eating Out,Purchase,NaN,TAMP & STEAM COFFEE HO LIVERSEDGE GB,NaN,NaN,NaN,NaN,NaN,11,23


In [150]:
labels = [label for label, idx in sorted(cat_map.items(), key=lambda x:x[1])] + ['in']

In [153]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    arrangement='perpendicular',
    node = dict(
      pad = 10,
      thickness = 10,
      line = dict(color = "black", width = 0.15),
      label = labels,
      color = "blue"
    ),
    link = dict(
      source = aug_dec_23['source'], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = aug_dec_23['target'],
      value = aug_dec_23['Amount'].apply(abs)
  ))])

fig.update_layout(title_text="Budget", font_size=20, width=2000, height = 1200)
fig.show()